In [12]:
import os, glob
import pandas as pd
import numpy as np

In [30]:
corpus_name = 'atc0'
root_path = '/project/graziul/ra/shiyanglai/corpus/'

def make_corpus_strcuture(root_path, corpus_name):
    if not os.path.exists(root_path + corpus_name):
        os.makedirs(root_path + corpus_name)
        # create three subfolders: lists, rttms, uems
        if not os.path.exists(root_path + corpus_name + '/lists'):
            os.makedirs(root_path + corpus_name + '/lists')
        else:
            print(f"Directory '{root_path + corpus_name + '/lists'}' already exist. Please delete it and try again.")

        if not os.path.exists(root_path + corpus_name + '/rttms'):
            os.makedirs(root_path + corpus_name + '/rttms')
            os.makedirs(root_path + corpus_name + '/rttms/train')
            os.makedirs(root_path + corpus_name + '/rttms/dev')
            os.makedirs(root_path + corpus_name + '/rttms/test')
        else:
            print(f"Directory '{root_path + corpus_name + '/rttms'}' already exist. Please delete it and try again.")

        if not os.path.exists(root_path + corpus_name + '/uems'):
            os.makedirs(root_path + corpus_name + '/uems')
            os.makedirs(root_path + corpus_name + '/uems/train')
            os.makedirs(root_path + corpus_name + '/uems/dev')
            os.makedirs(root_path + corpus_name + '/uems/test')
        else:
            print(f"Directory '{root_path + corpus_name + '/uems'}' already exist. Please delete it and try again.")

        print('The structure of corpus has been set successfully.')
    else:
        print(f"Directory '{root_path + corpus_name}' already exist. Please delete it and try again.")

In [21]:
# split train, development, test files
def split_atc0(train_rate=0.6, dev_rate=0.2):
    all_files = glob.glob('/project/graziul/data/corpora/atc0_comp/*/data/audio/*')
    all_files = [f for f in all_files if f.endswith('.sph') or f.endswith('.wav')]
    # random sampling
    train = all_files[:int(len(all_files)*train_rate)]
    devlopment = all_files[len(train):len(train)+int(len(all_files)*0.2)]
    test = all_files[len(train)+len(devlopment):]
    # inform the spliting results
    print('Corpus splitting successfully.')
    print(f'Training sample: {len(train)}.\nDevelopment sample: {len(devlopment)}.\nTesting sample: {len(test)}.')
    return train, devlopment, test

In [28]:
# generate lists
def generate_list(root_path, train, development, test):
    for name, data in zip(['train.txt', 'dev.txt', 'test.txt'], [train, development, test]):
        with open(root_path + '/lists/' + name, 'w') as f:
            uris = [audio.split('/')[-1].split('.')[0] for audio in data]
    print('.lst files are created successfully.')

In [ ]:
# generate rttms
def generate_rttms(root_path, train, development, test, reference):
    utt = pd,read_csv(reference)
    

In [29]:
train, dev, test = split_atc0()
generate_list(root_path='/project/graziul/ra/shiyanglai/corpus/atc0', train=train, development=dev, test=test)

Corpus splitting successfully.
Training sample: 31.
Development sample: 10.
Testing sample: 11.
.lst files are created successfully.


In [32]:
data = pd.read_csv('/project/graziul/data/corpora/atc0_comp/atc0.csv')

In [ ]:
data[data]

In [36]:
data.filePath

0        atc0_bos/data/audio/log_f1_1.sph
1        atc0_bos/data/audio/log_f1_1.sph
2        atc0_bos/data/audio/log_f1_1.sph
3        atc0_bos/data/audio/log_f1_1.sph
4        atc0_bos/data/audio/log_f1_1.sph
                       ...               
30432    atc0_dfw/data/audio/dfw_lw_3.sph
30433    atc0_dfw/data/audio/dfw_lw_3.sph
30434    atc0_dfw/data/audio/dfw_lw_3.sph
30435    atc0_dfw/data/audio/dfw_lw_3.sph
30436    atc0_dfw/data/audio/dfw_lw_3.sph
Name: filePath, Length: 30437, dtype: object